<a href="https://colab.research.google.com/github/NotARectangle/Honours2021/blob/main/AltModelTraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

In [2]:
import torch
torch.cuda.is_available()

True

In [4]:
#import torch

from transformers import GPT2Tokenizer, GPT2DoubleHeadsModel

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2DoubleHeadsModel.from_pretrained('gpt2')

tokenizer.add_special_tokens({"bos_token": "<bos>", "eos_token": "<eos>", "pad_token": "<pad>"})
model.resize_token_embeddings(len(tokenizer))

Some weights of GPT2DoubleHeadsModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.0.attn.masked_bias', 'h.1.attn.masked_bias', 'h.2.attn.masked_bias', 'h.3.attn.masked_bias', 'h.4.attn.masked_bias', 'h.5.attn.masked_bias', 'h.6.attn.masked_bias', 'h.7.attn.masked_bias', 'h.8.attn.masked_bias', 'h.9.attn.masked_bias', 'h.10.attn.masked_bias', 'h.11.attn.masked_bias', 'lm_head.weight', 'multiple_choice_head.summary.weight', 'multiple_choice_head.summary.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Embedding(50260, 768)

Train on Textfile.

extract data from json

In [5]:
import json
import re
import numpy as np
from sklearn.model_selection import train_test_split

data = json.load(open('drive/MyDrive/Honours2021/all_series_lines.json', 'r'))

In [7]:

script = {}

s_name = "TNG"
characters = ["PICARD", "TROI", "DATA", "RIKER"]
script = {"PICARD":[], "TROI":[], "DATA": [], "RIKER": []}
data = data[s_name]
episodes = data.keys()
for c in characters:
  for ep in episodes:
    for line in data[ep][c]:
      #add bos token, persona, line and then eos token at end.
      script[c].append(tokenizer.bos_token + " " + c + ": " + line + " " + tokenizer.eos_token)

In [8]:
def build_text_files(data_json, dest_path):
  f = open(dest_path, "w", encoding="utf-8")
  data = ""
  for block in data_json:
    for texts in block:
      summary = str(texts).strip()
      data += summary + " "
  print(str(len(data)))
  f.write(data)

In [9]:
train = []
test = []
for c in characters:
  c_train, c_test = train_test_split(script[c], test_size=0.20)
  train.append(c_train)
  test.append(c_test)

build_text_files(train,'train_dataset.txt')
build_text_files(test,'test_dataset.txt')

print("Train dataset length: "+str(len(train)))
print("Test dataset length: "+ str(len(test)))

2015479
506471
Train dataset length: 4
Test dataset length: 4


Training and datasetBuilder

In [10]:
from transformers import GPT2Tokenizer, GPT2DoubleHeadsModel

train_path = 'train_dataset.txt'
test_path = 'test_dataset.txt'
model_path = "drive/MyDrive/Honours2021/TNGv2"

tokenizer = GPT2Tokenizer.from_pretrained(model_path)
model = GPT2DoubleHeadsModel.from_pretrained(model_path)

from transformers import TextDataset, DataCollatorForLanguageModeling


def load_dataset(train_path, test_path, tokenizer):
    train_dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=train_path,
        block_size=128)

    test_dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=test_path,
        block_size=128)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset, test_dataset, data_collator

train_dataset, test_dataset, data_collator = load_dataset(train_path, test_path, tokenizer)


/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_mlm.py
  FutureWarning,


In [11]:

from transformers import Trainer, TrainingArguments


training_args = TrainingArguments(
    output_dir='./results',          # output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=64,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved 
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
)


trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,
    data_collator=data_collator,            # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset             # evaluation dataset
)

trainer.train()

trainer.save_model("./TNG")

Step,Training Loss


test

In [12]:

from transformers import pipeline

makeItSo = pipeline('text-generation',model='./TNG', tokenizer=tokenizer,config={'max_length':1200})

Some weights of the model checkpoint at ./TNG were not used when initializing GPT2LMHeadModel: ['multiple_choice_head.summary.weight', 'multiple_choice_head.summary.bias']
- This IS expected if you are initializing GPT2LMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2LMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:
makeItSo("PICARD: make it so.")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "PICARD: make it so.It is now and never was what it is. I cannot help thinking of her as one of my own, herself only.I think we should all have an idea. Let's bring the crew together because they"}]

In [14]:
makeItSo("PICARD: You will agree, Data, that Starfleet's orders are difficult? DATA: Difficult? Simply solve the mystery of Farpoint Station. PICARD:")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "PICARD: You will agree, Data, that Starfleet's orders are difficult? DATA: Difficult? Simply solve the mystery of Farpoint Station. PICARD:Ensign.We'll send our own probe on that mission.Let's hope they find the Rom"}]

In [17]:
makeItSo("PICARD: Would you object to your captain. RIKER:")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "PICARD: Would you object to your captain. RIKER:Aye, sir.And I know why, sir.If it is, we can't afford it.That does not appear to be your intent, Ambassador.I've not listened"}]

In [18]:
makeItSo("DATA: Can I ask you a question? PICARD: ")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'DATA: Can I ask you a question? PICARD: Yes, of course.Yes, Number One.You?No. Nothing, no, no. He thinks they are all dead. And then he gets distracted again, and they disappear.'}]

In [19]:
makeItSo("Riker: Can I ask you a question? TROI: ")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "Riker: Can I ask you a question? TROI: If you have any doubts, don't worry. This is Commander Riker.Doctor, Doctor. Please tell himYou know who you are.What are we to find?"}]

In [20]:
makeItSo("TROI:")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "TROI:, are you?How long have you been away?Will I be home soon?Riker, the Romulan warships are in formation.All right, I'll handle this. Let's go get the hell out of"}]

In [15]:
trainer.save_model("drive/MyDrive/Honours2021/TNGALTvs2")

In [16]:
tokenizer.save_pretrained("drive/MyDrive/Honours2021/TNGALTvs2")

('drive/MyDrive/Honours2021/TNGALTvs2/tokenizer_config.json',
 'drive/MyDrive/Honours2021/TNGALTvs2/special_tokens_map.json',
 'drive/MyDrive/Honours2021/TNGALTvs2/vocab.json',
 'drive/MyDrive/Honours2021/TNGALTvs2/merges.txt',
 'drive/MyDrive/Honours2021/TNGALTvs2/added_tokens.json')

In [22]:

from transformers import pipeline, GPT2Tokenizer

modelPath = "drive/MyDrive/Honours2021/TNGALTvs2"
makeItSo = pipeline('text-generation',model=modelPath, tokenizer=modelPath,config={'max_length':1200})

Some weights of the model checkpoint at drive/MyDrive/Honours2021/TNGALTvs2 were not used when initializing GPT2LMHeadModel: ['multiple_choice_head.summary.weight', 'multiple_choice_head.summary.bias']
- This IS expected if you are initializing GPT2LMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2LMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [23]:
makeItSo("USER: What do you think of this? PICARD:")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "USER: What do you think of this? PICARD:Why?Where are we? Our minds are on a mission to findthe truth.Mister Data, let's get right on it.Well done.Mister Worf"}]

In [24]:
makeItSo("RIKER: What do you think of this sir? PICARD:")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "RIKER: What do you think of this sir? PICARD:That does not stop me.He's getting into trouble. I have no choice but to help him.Mister La Forge, send an away team to theBridge.Thank you"}]